## Compulsory Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [1]:
# Load up store_income_data.csv

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.3 MB/s eta 0:00:00


In [4]:
from fuzzywuzzy import fuzz, process
import chardet
import pandas as pd
import numpy as np

In [5]:
worksheet = gc.open("store_income_data_task").sheet1
from gspread_dataframe import get_as_dataframe

df = get_as_dataframe(worksheet)

1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [6]:
print("Original 'country' values:")
print(df['country'].unique())

df['country'] = df['country'].str.lower().str.strip()

print("\nCleaned 'country' values:")
print(df['country'].unique())

Original 'country' values:
['United States/' 'Britain' 'United States' 'Britain/' 'United Kingdom'
 'U.K.' 'SA' 'U.K/' 'America' nan 'united states' 'S.A.' 'England' 'UK'
 'S.A./' 'ENGLAND' 'BRITAIN' 'U.K' 'America/' 'SA.' 'u.k' 'uk' 'UK.'
 'England/' 'england' 'united states of america' 'UK/' 'SA/' 'England.'
 'UNITED KINGDOM' 'America.' 'S.A..' 's.a.' 'United States of America'
 'United States of America.' 'United States of America/' 'United States.'
 's. africasouth africa' 'united kingdom' 'AMERICA'
 'UNITED STATES OF AMERICA' 'S. AfricaSouth Africa' 'america'
 'S. AFRICASOUTH AFRICA' 'Britain.' '/' 'United Kingdom.' 'UNITED STATES'
 'sa' 'S. AfricaSouth Africa/' 'United Kingdom/' 'S. AfricaSouth Africa.'
 '.' 'britain']

Cleaned 'country' values:
['united states/' 'britain' 'united states' 'britain/' 'united kingdom'
 'u.k.' 'sa' 'u.k/' 'america' nan 's.a.' 'england' 'uk' 's.a./' 'u.k'
 'america/' 'sa.' 'uk.' 'england/' 'united states of america' 'uk/' 'sa/'
 'england.' 'america.'

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [9]:
# Converting to string, lowercase, strip
df['country'] = df['country'].astype(str).str.lower().str.strip()

# Defining the valid country names
standard_countries = ['south africa', 'united kingdom', 'united states']

# Function with safety check for non-empty strings
def match_country(name):
    if name == '' or name == 'nan':
        return name
    match, score = process.extractOne(name, standard_countries)
    if score >= 75:
      return match

    if name in ['usa','united states of america']:
      return 'united states'
    elif name in ['uk','u.k.','united kingdom(uk)']:
      return 'united kingdom'
    elif name in ['south africa', 's.a.']:
      return 'south africa'
    else:
      return name

# Applying fuzzywuzzy matching
df['country'] = df['country'].apply(match_country)

print("Final cleaned country values:")
print(df['country'].unique())



Final cleaned country values:
['united states' 'britain' 'britain/' 'united kingdom' 'sa' 'u.k/'
 'america' 'nan' 'south africa' 'england' 'u.k' 'america/' 'sa.' 'uk.'
 'england/' 'uk/' 'sa/' 'england.' 'america.' 'britain.' '/' '.']


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [10]:
from datetime import datetime, date

df['date_measured'] = pd.to_datetime(df['date_measured'], errors='coerce')

df = df[df['date_measured'].notna()]

today = pd.Timestamp(date.today())

df['days_ago'] = (today - df['date_measured']).dt.days

df[['date_measured', 'days_ago']].head()


<ipython-input-10-5580f623416e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_ago'] = (today - df['date_measured']).dt.days


,date_measured,days_ago
0,2006-04-02,6932
1,2006-04-01,6933
2,2003-12-09,7777
3,2006-08-05,6807
6,2001-08-06,8632
